In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q ltp

     |████████████████████████████████| 1.8MB 5.3MB/s 
     |████████████████████████████████| 890kB 24.0MB/s 
     |████████████████████████████████| 2.9MB 28.0MB/s 
     |████████████████████████████████| 163kB 6.4MB/s 
     |████████████████████████████████| 245kB 16.2MB/s 
     |████████████████████████████████| 20.7MB 1.7MB/s 
     |████████████████████████████████| 81kB 3.1MB/s 


In [2]:
!git clone https://github.com/Muennighoff/wecredo.git

Cloning into 'wecredo'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 84 (delta 38), reused 66 (delta 20), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [3]:
import tensorflow as tf
import math
import transformers
from transformers import BertConfig, RobertaConfig
from datasets import load_dataset

import os

from wecredo.tokenization import WWMTokenizer
from wecredo.modeling import TFRobertaForMaskedLM

In [4]:
# If this cell takes very long to run; factory reset and run this cell the very first

tf.keras.backend.clear_session()

try:
    # Locate TPU
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    # Establish communication / Locates TPU on the network
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_WORKER)
    # Connect
    tf.config.experimental_connect_to_cluster(resolver)
    # Start TPU
    tf.tpu.experimental.initialize_tpu_system(resolver)
    # Set strategy
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

    print("Number of accelerators: ", strategy.num_replicas_in_sync)

except ValueError:
    print("No TPU found")

# Check devices
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.24.47.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.24.47.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Number of accelerators:  8
All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [5]:
### Modelling - Static ###

class Wrapper(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        # Track loss (Loss itself its CategoricalCrossEnt.)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss') 

    @tf.function
    def train_step(self, data):
        # Data is a dictionary
        y = data["lm_label_ids"]
        x = data

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.model(x, training=True)
            # Compute the loss value.
            y_pred = y_pred.logits
            loss = self.compute_loss(y, y_pred)
            # Reduce loss to single digit
            loss = tf.reduce_mean(loss)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update loss tracker
        self.loss_tracker.update_state(loss)  
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Data is a dictionary
        y = data["lm_label_ids"]
        x = data

        # Compute predictions
        y_pred = self.model(x, training=False)
        y_pred = y_pred.logits
        loss = self.compute_loss(y, y_pred)

        # Updates the metrics tracking the loss
        self.loss_tracker.update_state(loss)  
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def compute_loss(self, labels, logits):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE
        )
        # make sure only labels that are not equal to -100 do affect loss
        active_loss = tf.not_equal(tf.reshape(labels, (-1,)), -100)
        reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
        labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

        return loss_fn(labels, reduced_logits)

In [6]:
def shape_list(tensor: tf.Tensor):
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (:obj:`tf.Tensor`): The tensor we want the shape of.
    Returns:
        :obj:`List[int]`: The shape of the tensor as a list.

    ### TODO: Move to compute_loss
    """
    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

#### TPU-Base

In [77]:
def create_model():
    """
    Creates Model
    """
    config = RobertaConfig.from_pretrained("roberta-base")
    config.performer = True
    base_model = TFRobertaForMaskedLM(config)
    model = Wrapper(base_model)
    return model


def tf_dataset_slices(train_ds, tokenizer):
    """
    Diff method of turning into TF Dataset
    """
    train_ds.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids'])
    #features = {x: train_ds[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask', 'token_type_ids']}
    #train_ds = tf.data.Dataset.from_tensor_slices((features, train_ds["lm_label_ids"]))

    features = {x: train_ds[x].to_tensor(default_value=0, shape=[None, tokenizer.model_max_length]) for x in ['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids']}
    train_ds = tf.data.Dataset.from_tensor_slices(features)

    return train_ds


def get_dataset(batch_size, is_training=True):
    """
    Only train for now;
    """
    train_ds = load_dataset('c3', 'mixed', split='train')
    def prepare_dict(example):
        example["text"] = example["documents"][0]
        return example
    train_ds = train_ds.map(prepare_dict)

   # train_ds = train_ds.select(range(32))

    tokenizer = WWMTokenizer(col="text", seq_len=512)
    train_ds = train_ds.map(tokenizer.tokenize_pretraining)
    train_ds = tf_dataset_slices(train_ds, tokenizer.tokenizer_cn)

    # Need to drop the remainder if len(ds) // bs != 0
    train_ds = train_ds.shuffle(1000).batch(batch_size, drop_remainder=True)

    return train_ds

In [78]:
# Process on TPU
with strategy.scope():
    model = create_model()

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  metrics=['sparse_categorical_accuracy'])


batch_size = 8
steps_per_epoch = 3138 // batch_size # == 523

train_dataset = get_dataset(batch_size, is_training=True)

model.fit(train_dataset,
          epochs=20,
          steps_per_epoch=steps_per_epoch)

Reusing dataset c3 (/root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c/cache-6cab0f82894ebe0f.arrow


Token indices sequence length is longer than the specified maximum sequence length for this model (943 > 512). Running this sequence through the model will result in indexing errors


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/20


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


392/392 [==============================] - 94s 96ms/step - sparse_categorical_accuracy: 0.0026 - loss: 8.6843
Epoch 2/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0028 - loss: 6.9976
Epoch 3/20
392/392 [==============================] - 38s 96ms/step - sparse_categorical_accuracy: 0.0032 - loss: 6.5932
Epoch 4/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0038 - loss: 6.3945
Epoch 5/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0044 - loss: 6.2587
Epoch 6/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0048 - loss: 6.1535
Epoch 7/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0054 - loss: 6.0618
Epoch 8/20
392/392 [==============================] - 38s 97ms/step - sparse_categorical_accuracy: 0.0056 - loss: 5.9854
Epoch 9/20
392/392 [=======================

In [79]:
### Evaluation ###

example = next(iter(train_dataset))
input_ids = example["input_ids"]

tokenizer = WWMTokenizer(col="text", seq_len=512)

print("Inputs: ", tokenizer.tokenizer_cn.decode(input_ids.numpy()[0]))

# Somehow model.model.predict returns None
logits = model.model(example).logits

tensor = tf.math.argmax(
              logits, axis=-1, output_type=tf.dtypes.int64, name=None
          )

preds = tokenizer.tokenizer_cn.decode(tensor[0])

print("Preds: ", preds)

# [PAD] are not predicted for 
x = tf.where(example["lm_label_ids"][0] > 0, example["lm_label_ids"][0], 0)
print("Solutions:",tokenizer.tokenizer_cn.decode(x))

Inputs:  [CLS] 看 地图 [MASK] 体育馆 在 图书馆 的 东边 ， 但是 我 刚才 问 了 几 个 人 ， 他们 都 [MASK] [MASK] [MASK]馆 早就 搬 了 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

#### TPU w/ an avg 2220 tokens / example

In [14]:
class IncreaseLen():
  def __init__(self):
      self.avg_len = 0
      self.count = 0

      self.last_examples = ''
  
  def concat(self, example):
      self.last_examples += example["text"]
      if len(self.last_examples) > (2048):  # Example must be longer than 2K
          example["text"] = self.last_examples
          self.last_examples = ''
          return True
      else:
          return False

  def average_length(self, example):
      self.avg_len += len(example["text"])
      self.count += 1

  def return_avg(self):
      return self.avg_len / self.count

In [15]:
def create_model():
    """
    Creates Model
    """
    config = RobertaConfig.from_pretrained("roberta-base")
    config.performer = True
    config.max_position_embeddings = 3000 # Need to incr. max emb. size, which is 512 by default in bert
    base_model = TFRobertaForMaskedLM(config)
    model = Wrapper(base_model)
    return model


def tf_dataset_slices(train_ds, tokenizer):
    """
    Diff method of turning into TF Dataset
    """
    train_ds.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids'])

    features = {x: train_ds[x].to_tensor(default_value=0, shape=[None, None]) for x in ['input_ids', 'attention_mask', 'token_type_ids', 'lm_label_ids']}
    train_ds = tf.data.Dataset.from_tensor_slices(features)

    return train_ds


def get_dataset(batch_size, is_training=True):
    """
    Only train for now;
    """
    train_ds = load_dataset('c3', 'mixed', split='train')
    def prepare_dict(example):
        example["text"] = example["documents"][0]
        return example
    train_ds = train_ds.map(prepare_dict)

    incr_len = IncreaseLen()
    train_ds = train_ds.filter(incr_len.concat)
    train_ds.map(incr_len.average_length)
    print("Average Length: ", incr_len.return_avg())
    
    #train_ds = train_ds.select(range(32)) #########

    tokenizer = WWMTokenizer(col="text", seq_len=2500) # Max Seq Len of 2500

    train_ds = train_ds.map(tokenizer.tokenize_pretraining)
    train_ds = tf_dataset_slices(train_ds, tokenizer.tokenizer_cn)

    # Need to drop the remainder if len(ds) // bs != 0
    train_ds = train_ds.shuffle(1000).batch(batch_size, drop_remainder=True)

    return train_ds

In [16]:
# Make sure we use BFloats instead of Floats32
!export XLA_USE_BF16=1

In [17]:
# Process on TPU
with strategy.scope():
    model = create_model()

    learning_rate = 1e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=optimizer,
                  metrics=['sparse_categorical_accuracy'])


batch_size = 8

train_dataset = get_dataset(batch_size, is_training=True)

model.fit(train_dataset,
          epochs=10)

# At len > 4096  &   BS 8: ResourceExhaustedError

Reusing dataset c3 (/root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/c3/mixed/1.0.0/6bcfb26ae1bd77bd57d300c2504900834cc29aaa092eb87c4d91d7960a3c2d8c/cache-a044a2ef1844e952.arrow



Average Length:  2221.654901960784


Token indices sequence length is longer than the specified maximum sequence length for this model (2049 > 512). Running this sequence through the model will result in indexing errors


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/10


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


31/31 [==============================] - 67s 291ms/step - sparse_categorical_accuracy: 0.0052 - loss: 10.1803
Epoch 2/10
31/31 [==============================] - 9s 293ms/step - sparse_categorical_accuracy: 0.0070 - loss: 9.6454
Epoch 3/10
31/31 [==============================] - 9s 292ms/step - sparse_categorical_accuracy: 0.0072 - loss: 9.2590
Epoch 4/10
31/31 [==============================] - 9s 291ms/step - sparse_categorical_accuracy: 0.0069 - loss: 8.8993
Epoch 5/10
31/31 [==============================] - 9s 291ms/step - sparse_categorical_accuracy: 0.0071 - loss: 8.5657
Epoch 6/10
31/31 [==============================] - 9s 292ms/step - sparse_categorical_accuracy: 0.0073 - loss: 8.2533
Epoch 7/10
31/31 [==============================] - 9s 292ms/step - sparse_categorical_accuracy: 0.0073 - loss: 7.9748
Epoch 8/10
31/31 [==============================] - 9s 292ms/step - sparse_categorical_accuracy: 0.0073 - loss: 7.7274
Epoch 9/10
31/31 [==============================] - 9s 29

In [18]:
### Evaluation ###

example = next(iter(train_dataset))
input_ids = example["input_ids"]

tokenizer = WWMTokenizer(col="text", seq_len=3000)

print("Inputs: ", tokenizer.tokenizer_cn.decode(input_ids.numpy()[0]))

# Somehow model.model.predict returns None
logits = model.model(example).logits

tensor = tf.math.argmax(
              logits, axis=-1, output_type=tf.dtypes.int64, name=None
          )

preds = tokenizer.tokenizer_cn.decode(tensor[0])

print("Preds: ", preds)

# [PAD] are not predicted for 
x = tf.where(example["lm_label_ids"][0] > 0, example["lm_label_ids"][0], 0)
print("Solutions:",tokenizer.tokenizer_cn.decode(x))

Inputs:  [CLS] 英国 的 研究 人员 [MASK]示 ， 糖 对 牙齿 [MASK] [MASK] 的 [MASK] [MASK] 会 [MASK] [MASK] 积累 并 伴随 一生 ， 要 保持 牙齿 健康 ， 每天 吃 糖 不 应该 超过 5 勺 。 研究 还walker [MASK] ， 尽管 牙膏 [MASK] 在 [MASK] 被 程度 上 保护 牙齿 ， 但 不 能 完 [MASK] 消除 日常 [MASK] [MASK] 中 糖 [MASK] 牙齿 造成 的 伤害 ， 即使 经常 刷牙 ， 吃 太 多 的 糖 还是 [MASK] 造成 蛀牙 [MASK] 参与 研究 的 教授 说 ， 以前 人们 只 在 特定 的 节日 吃 糖 ， [MASK] [MASK] [MASK] 含 糖 的 食物 和 饮料 成 了 人们 [MASK] 主要 食物 ， 而 过量 吃 糖 会 影响 牙齿 [MASK] 心脏 健康 。 怎样 [MASK] [MASK] 哪些 书 是 值得 精读 的 呢 [MASK] 自 [MASK] [MASK] [MASK] ， 已经 有 一 位 最 公正 的 评选家 ， [MASK] [MASK] [MASK] 推荐者 向 [MASK] 推荐 好 书 。 习 sunny 评选家 就是 时间 ， 这些 推荐者 就是 读者 [MASK] 历代 宮 读者 ， 把 他们 认为 有 价值 的 [MASK] 推荐 给 时间 ， 时间 照 着 他们 的 推荐 ， 把 那些 没有 永久 价值 的 书 都 刷 下去 ， 把 那些 有 永久 价值 的 书 都 流传 discuz [MASK] 。 我们 [MASK]層 古代 流传 下来 仆 书 ， 61 部分 都 是 有 价值 的 ， 心里 会 觉得 奇怪 ， 怎么 古人 写 的 [MASK] [MASK] [MASK] 是 有 价值 [MASK]? [MASK] [MASK] ， 他们 所 写 的 东西 也 有 许多 没有 价值 的 ， 不过 ， 那些 没有 [MASK] [MASK] 的 东西 ， [MASK] [MASK] 为 历代 读者 所 推荐 ， 在 时 [MASK] 的 考验 中 落选 了 。 现在 [MASK] [MASK] 所谓 的 [UNK] 经典著作 [UNK] 都 是 经过 